<a href="https://colab.research.google.com/github/pythagoras-yamamoto/colab-vscode-git-zsh-env/blob/main/vscode_colab_env.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Google Colab proをVSCodeでssh接続してV100(GPU)を早速回してみた。

https://qiita.com/Never_/items/3dfcc8cf5c2765832c74

Password、Portは毎回可変のためVSCodeのSSH Targetに適宜設定する

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import random, string, urllib.request, json, getpass

In [14]:
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))
password

'1ASoMT1KyYxNufnXdyxF'

In [15]:
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

In [16]:
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

In [17]:
#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

In [19]:
get_ipython().system_raw('/usr/sbin/sshd -D &')

In [20]:
#Access Token 
authtoken="2DW5JK919gPbTATMozj0z57E2z1_3bLLtmn4n6MuXeTCuPCsR"

In [21]:
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

In [22]:
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  (host, port) = data['tunnels'][0]['public_url'][6:].split(':')
  print(f'SSH command: ssh -p{port} root@{host}')

print(f'Root password: {password}')

SSH command: ssh -p17906 root@6.tcp.ngrok.io
Root password: 1ASoMT1KyYxNufnXdyxF
